Install Dependencies

In [ ]:
%pip install torch torchvision
%pip install --upgrade torch
%pip install --upgrade torchvision
%pip install tensorflow
%pip install torchsummary

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchsummary import summary

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time


# Dataset
class PokemonDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
    @property
    def classes(self):
        return self.data.classes

transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
data_dir = "'..\data\separated_imgdata'"

dataset = PokemonDataset(data_dir, transform)
image, label = dataset[100]
image.shape


# DataLoader
train_size = int(0.7 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
testloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

class Model(nn.Module):
    def __init__(self, num_classes = len(dataset.classes)):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, (3, 3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3, 3)),
            nn.ReLU(),
            nn.Conv2d(64, 128, (3, 3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(128 * (224-6) * (224-6), num_classes)
        )
    
    # def classifier()

    def forward(self, x):
        return self.model(x)
    
model = Model().to('cuda')
loss_func = nn.CrossEntropyLoss()
optimiser = optim.SGD(model.parameters(), lr=0.001, momentum=0.9,)

summary(model, (3, 224, 224))

exp_lr_scheduler = lr_scheduler.StepLR(optimiser, step_size=2, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0
    best_score = np.inf
    patience = 5

    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            dataloader = trainloader if phase == 'train' else testloader
            for inputs, labels in dataloader:
                inputs = inputs.to('cuda')
                labels = labels.to('cuda')

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()
                for param_group in optimizer.param_groups:
                    print(f'Learning Rate: {param_group["lr"]}')

            epoch_loss = running_loss / len(train_dataset if phase == 'train' else test_dataset)
            epoch_acc = running_corrects.double() / len(train_dataset if phase == 'train' else test_dataset)

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()
            
        # Convergence Check
        if(epoch_loss < best_score):
            best_score = epoch_loss
            print(f'Patience: {patience}')    
        
        else:
            patience -= 1
            print(f'Patience: {patience}')
        
        if patience <= 0:
            print(f"EARLY STOP at EPOCH #{epoch}")
            break

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

model_ft = train_model(model, loss_func, optimiser, exp_lr_scheduler, num_epochs=25)

In [ ]:
from sklearn.metrics import classification_report
import torch

model_ft.eval()

all_labels = []
all_preds = []

with torch.no_grad(): 
    for inputs, labels in testloader:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')

        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
class_names = dataset.classes
report = classification_report(all_labels, all_preds, target_names=class_names, zero_division=0)

print(report)